In [1]:
%matplotlib inline
import collections
import math
import os
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

import re
import pickle
import pandas as pd
import csv
import os
import time
import datetime
from tensorflow.contrib import learn
from nltk.tokenize import TweetTokenizer
from sklearn.metrics import f1_score
from IPython.display import clear_output, Image, display, HTML

In [2]:
###### Do not modify here ###### 
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = graph_def
    #strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
###### Do not modify  here ######

In [3]:
def load_data_and_labels(train_data_file, test_data_file):
    """
    Loads MR polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """
    # Load data from files
    train_data = pd.read_csv(train_data_file, sep='\t', quoting=csv.QUOTE_NONE, header=None, names=['id', 'language', 'label', 'text'])
    test_data = pd.read_csv(test_data_file, sep='\t', quoting=csv.QUOTE_NONE, header=None, names=['id', 'language', 'label', 'text'])
    
    x_train = train_data['text'].tolist()
    y_train = train_data['label'].tolist()

    x_test = test_data['text'].tolist()
    y_test = test_data['label'].tolist()
    
    x_train = [s.strip() for s in x_train]
    x_test = [s.strip() for s in x_test]
    
    label_encoding = {'neutral':0, 'positive':1, 'negative':2}
    
    y_train_encoding = [label_encoding[label] for label in y_train]    
    y_test_encoding = [label_encoding[label] for label in y_test]

    
    return [x_train, y_train_encoding, x_test, y_test_encoding]

def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        print("Current epoch: ", epoch)
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [4]:
final_embeddings = np.load('./data/embed_tweets_en_200M_200D/embedding_matrix.npy')
word_dict = {}
with open('./data/embed_tweets_en_200M_200D/vocabulary.pickle', 'rb') as myfile:
    word_dict = pickle.load(myfile)

In [5]:
# shit
for key, val in word_dict.items():
    if val[0] == 94:
        print(key, val)

> (94, 3868680)


In [6]:
final_embeddings.shape

(1859185, 200)

In [7]:
#Load label data
x_train_sentence, y_train, x_test_sentence, y_test = load_data_and_labels('./data/supervised_data/en_full.tsv.txt', './data/supervised_data/en_test.tsv')
# max_document_length = max([len(x.split(" ")) for x in x_train_sentence])
# print(max_document_length)

In [8]:
len(x_test_sentence)

20632

In [9]:
"""
    This function assumes that the last word in the word embedding is a zero vector, and will use it as padding.
    The input 'num_voc' equals to the shape[0] of the word embedding.
"""
def process_tweet(train_tweets, test_tweets, num_voc):
    # max_document_length = max([len(x.split(" ")) for x in x_train_sentence])
    ppl_re = re.compile(r'@\S*')
    url_re = re.compile(r'http\S+')
    tknzr = TweetTokenizer()
    # tknzr = TweetTokenizer(reduce_len=True)
    
    tokenized_tweets_all = []
    max_document_length = 0
    
    for tweets in [train_tweets, test_tweets]:
        tweets = [url_re.sub('URLTOK', ppl_re.sub('USRTOK', tweet.lower())) for tweet in tweets]
        tokenized_tweets = [tknzr.tokenize(tweet) for tweet in tweets]
        tokenized_tweets_all.append(tokenized_tweets)
        max_document_length = max(max_document_length, max([len(tweet) for tweet in tokenized_tweets]))
    print(max_document_length)
    
    x = []
    
    for tokenized_tweets in tokenized_tweets_all:
        x_curr = []
        for tokenized_tweet in tokenized_tweets:
            if len(tokenized_tweet) == max_document_length:
                print(tokenized_tweet)
            """Not sure if original paper does this, but since index 0 means USRTOK, padding should be a number
            higher than total word count, so tf.nn.embedding_lookup will return a tensor of 0 insted of USRTOK."""
        #     temp = np.zeros(max_document_length, dtype=np.int).tolist()
            temp = (np.ones(max_document_length, dtype=np.int)*(num_voc-1)).tolist()

            for index, word in enumerate(tokenized_tweet):
                if word in word_dict:
                    temp[index] = word_dict[word][0]
            x_curr.append(temp)
        x_curr = np.array(x_curr)
        x.append(x_curr)
    
    return x[0], x[1]

x_train, x_test = process_tweet(x_train_sentence, x_test_sentence, final_embeddings.shape[0])

53
["there's", 'a', 'lot', 'of', 'stupid', '$', 'h', '!', 't', 'out', 'there', ',', 'but', 'polling', 'trump', 'v', 'kanye', 'west', 'may', 'take', 'the', 'cake', '.', 'all', 'i', 'can', 'think', 'to', 'say', 'is', ':', '#', '$', '%', '#', '$', '%', '$', '#', '%', '#', '$', '%', '#', '$', '%', '#', '$', '#', '$', '#', '$', '%']


In [10]:

# x_train = []

# for tweet in x_train_sentence:
#     """Not sure if original paper does this, but since index 0 means USRTOK, padding should be a number
#     higher than total word count, so tf.nn.embedding_lookup will return a tensor of 0 insted of USRTOK."""
# #     temp = np.zeros(max_document_length, dtype=np.int).tolist()
#     temp = (np.ones(max_document_length, dtype=np.int)).tolist()
    
#     # shitty paper preprocessing
#     tweet = tweet.lower()
#     tweet = ppl_re.sub('USRTOK', tweet)
#     tweet = url_re.sub('URLTOK', tweet)
    
#     for index, word in enumerate(tknzr.tokenize(tweet)):
#         if word in word_dict:
#             temp[index] = word_dict[word][0]
#         else:
#             print(word)
#             break
#     x_train.append(temp)
# x_train = np.array(x_train)

In [11]:
# put Word2Vec on 590 million English Tweets using 52 dimensions.
vocabulary_size = final_embeddings.shape[0]
embedding_size = 200  # Dimension of the embedding vector.
graph = tf.Graph()

sequence_length=x_train.shape[1]
num_classes=3

# filter_sizes: The number of words we want our convolutional filters to cover. 
# We will have num_filters for each size specified here. 
# For example, [3, 4, 5] means that we will have filters that slide over 3, 4 and 5 words respectively, for a total of 3 * num_filters filters.
first_filter_sizes = [4]
first_pool_window_sizes = [4]
first_pool_strides = [2]


second_filter_window_sizes = [3]
num_filters = 200

# No L2 norm
l2_reg_lambda=0.0

with graph.as_default():
    input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
    input_y = tf.placeholder(tf.int64, [None], name="input_y")
    with tf.device('/cpu:0'):
        embeddings = tf.Variable(tf.constant(0.0, shape=[vocabulary_size, embedding_size]),
                        trainable=False, name="embedding")

        embedding_placeholder = tf.placeholder(tf.float32, [vocabulary_size, embedding_size], name='word_embedding_placeholder')
        embedding_init = embeddings.assign(embedding_placeholder)  # assign exist word embeddings

        embedded_chars = tf.nn.embedding_lookup(embeddings, input_x)
    embedded_chars_expanded = tf.expand_dims(embedded_chars, -1)
    print(input_x.shape)
    print(embedded_chars_expanded.shape)
    # Keeping track of l2 regularization loss (optional)
    l2_loss = tf.constant(0.0)
    
     # Add variable initializer.
    init = tf.global_variables_initializer()
    
    
    # Create first cnn : a convolution + maxpool layer for each filter size    
    # 1st Convolution Layer
    for i, first_filter_size in enumerate(first_filter_sizes):
        with tf.name_scope("conv-maxpool-1"):
            # Convolution Layer
            filter_shape = [first_filter_size, embedding_size, 1, num_filters]
            W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
            print("CNN filter", W.shape)
            b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
            conv = tf.nn.conv2d(
                embedded_chars_expanded,
                W,
                strides=[1, 1, 1, 1],
                padding="VALID",
                name="conv")
            # Apply nonlinearity
            h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
            # Maxpooling over the outputs and transform for next layer, so the "channel" of convolution 
            # will become "input_width" for next layer
#             pooled = tf.transpose(tf.nn.max_pool(
#                 h,
#                 ksize=[1, first_pool_window_sizes[i], 1, 1],
#                 strides=[1, first_pool_strides[i], 1, 1],
#                 padding='VALID',
#                 name="pool"), perm=[0, 1, 3, 2])
            pooled = tf.nn.max_pool(
                h,
                ksize=[1, first_pool_window_sizes[i], 1, 1],
                strides=[1, first_pool_strides[i], 1, 1],
                padding='VALID',
                name="pool")
#     print("conv1", conv.shape)
#     print("h1", h.shape)
#     print("pooled1", pooled_1.shape)
    
    # 2nd Convolutional Layer
#     for i, second_filter_size in enumerate(second_filter_window_sizes):
#         with tf.name_scope("conv-maxpool-2"):
#             # Convolution Layer
#             filter_shape = [second_filter_size, num_filters, 1, num_filters]
#             W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
#             print("CNN filter", W.shape)
#             b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
#             conv = tf.nn.conv2d(
#                 pooled,
#                 W,
#                 strides=[1, 1, 1, 1],
#                 padding="VALID",
#                 name="conv")
#             # Apply nonlinearity
#             h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
#             # Maxpooling over the outputs and transform for next layer, so the "channel" of convolution 
#             # will become "input_width" for next layer
#             pooled = tf.nn.max_pool(
#                 h,
#                 ksize=[1, h.shape[1], 1, 1],
#                 strides=[1, 1, 1, 1],
#                 padding='VALID',
#                 name="pool")
    for i, second_filter_size in enumerate(second_filter_window_sizes):
        with tf.name_scope("conv-maxpool-2"):
            # Convolution Layer
            filter_shape = [second_filter_size, 1, num_filters, num_filters]
            W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
            print("CNN filter", W.shape)
            b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
            conv = tf.nn.conv2d(
                pooled,
                W,
                strides=[1, 1, 1, 1],
                padding="VALID",
                name="conv")
            # Apply nonlinearity
            h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
            print(h.shape)
            # Maxpooling over the outputs and transform for next layer, so the "channel" of convolution 
            # will become "input_width" for next layer
            pooled = tf.nn.max_pool(
                h,
                ksize=[1, h.shape[1], 1, 1],
                strides=[1, 1, 1, 1],
                padding='VALID',
                name="pool")
 

    h_pool_flat = tf.reshape(pooled, [-1, num_filters])  # flatten pooling layers
    print("h_pool_flat", h_pool_flat.shape)
    
    # Add dropout
#     with tf.name_scope("dropout"):
#         self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

    # Final (unnormalized) scores and predictions
    
    # Fully connected hidden layer
    with tf.name_scope("hidden"):
        with tf.variable_scope("hidden"):
            W = tf.get_variable(
                "W",
                shape=[num_filters, num_filters],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            out = tf.nn.relu(tf.nn.xw_plus_b(h_pool_flat, W, b))
        
    
    with tf.name_scope("output"):
        with tf.variable_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            scores = tf.nn.xw_plus_b(out, W, b, name="scores")
            print("scores", scores.shape)
            predictions = tf.argmax(scores, 1, name="predictions")
            print("predictions", predictions.shape)


    # Calculate mean cross-entropy loss
    with tf.name_scope("loss"):
        losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=scores, labels=input_y)
        print("losses", losses.shape)
        loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

    # Accuracy
    with tf.name_scope("accuracy"):
        correct_predictions = tf.equal(predictions, input_y)
        accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")
    

(?, 53)
(?, 53, 200, 1)
CNN filter (4, 200, 1, 200)
CNN filter (3, 1, 200, 200)
(?, 22, 1, 200)
h_pool_flat (?, 200)
scores (?, 3)
predictions (?,)
losses (?,)


In [12]:
show_graph(graph.as_graph_def())

In [ ]:
batch_size = 1024
num_checkpoints = 5
num_epochs = 2000
print_train_every = 5
evaluate_every = 50
checkpoint_every = 10000000
allow_soft_placement=True
log_device_placement=False

with graph.as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=allow_soft_placement,
      log_device_placement=log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        
        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdadeltaOptimizer(1.0)
        grads_and_vars = optimizer.compute_gradients(loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", loss)
        acc_summary = tf.summary.scalar("accuracy", accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=num_checkpoints)

#         # Write vocabulary
#         vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())


        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              input_x: x_batch,
              input_y: y_batch,
            }
            _, step, summaries, cur_loss, cur_accuracy, y_pred = sess.run(
                [train_op, global_step, train_summary_op, loss, accuracy, predictions],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
#             print(y_pred)
#             print(y_batch)
            if step % print_train_every == 0:
                f1 = f1_score(y_batch, y_pred, average = 'weighted')
                print("{}: step {}, loss {:g}, acc {:g}, f1 {:g}".format(time_str, step, cur_loss, cur_accuracy,
                                                                     f1))
            train_summary_writer.add_summary(summaries, step)



        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              input_x: x_batch,
              input_y: y_batch,
            }
            step, summaries, cur_loss, cur_accuracy, y_pred = sess.run(
                [global_step, dev_summary_op, loss, accuracy, predictions],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            f1 = f1_score(y_batch, y_pred, average = 'weighted')
            print("{}: step {}, loss {:g}, acc {:g}, f1 {:g}".format(time_str, step, cur_loss, cur_accuracy,
                                                                    f1))
            if writer:
                writer.add_summary(summaries, step)

        
        sess.run(embedding_init, feed_dict={embedding_placeholder: final_embeddings})
        # Generate batches
        batches = batch_iter(
            list(zip(x_train, y_train)), batch_size, num_epochs)
        
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            
            current_step = tf.train.global_step(sess, global_step)
            if current_step % evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_test, y_test, writer=dev_summary_writer)
                print("")
            if current_step % checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))

INFO:tensorflow:Summary name conv-maxpool-1/W:0/grad/hist is illegal; using conv-maxpool-1/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-1/W:0/grad/sparsity is illegal; using conv-maxpool-1/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-1/b:0/grad/hist is illegal; using conv-maxpool-1/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-1/b:0/grad/sparsity is illegal; using conv-maxpool-1/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-2/W:0/grad/hist is illegal; using conv-maxpool-2/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-2/W:0/grad/sparsity is illegal; using conv-maxpool-2/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-2/b:0/grad/hist is illegal; using conv-maxpool-2/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-2/b:0/grad/sparsity is illegal; using conv-maxpool-2/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name hidden/W:0/grad/hist is ill

C:\Python36\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


2017-11-15T22:50:22.687672: step 5, loss 1.30267, acc 0.407227, f1 0.235688
2017-11-15T22:50:23.229151: step 10, loss 1.03652, acc 0.435547, f1 0.324913
2017-11-15T22:50:23.777150: step 15, loss 0.983748, acc 0.470703, f1 0.428884
Current epoch:  1
2017-11-15T22:50:24.292058: step 20, loss 1.04705, acc 0.447266, f1 0.319714
2017-11-15T22:50:24.806651: step 25, loss 1.01988, acc 0.444336, f1 0.329845
2017-11-15T22:50:25.318222: step 30, loss 0.951115, acc 0.515625, f1 0.478291
2017-11-15T22:50:25.832428: step 35, loss 0.937164, acc 0.520508, f1 0.467849
Current epoch:  2
2017-11-15T22:50:26.317045: step 40, loss 0.986472, acc 0.476562, f1 0.362407
2017-11-15T22:50:26.827950: step 45, loss 1.03645, acc 0.507812, f1 0.412241
2017-11-15T22:50:27.357403: step 50, loss 0.929724, acc 0.536133, f1 0.502296

Evaluation:
2017-11-15T22:50:28.061947: step 50, loss 0.947633, acc 0.50475, f1 0.481905

Current epoch:  3
2017-11-15T22:50:28.557343: step 55, loss 0.946152, acc 0.508789, f1 0.438146
201

2017-11-15T22:51:14.098938: step 450, loss 0.880567, acc 0.578567, f1 0.516643

Current epoch:  25
2017-11-15T22:51:14.623460: step 455, loss 0.696214, acc 0.689453, f1 0.685811
2017-11-15T22:51:15.140942: step 460, loss 0.69546, acc 0.693359, f1 0.674622
2017-11-15T22:51:15.652510: step 465, loss 0.763183, acc 0.642578, f1 0.655698
Current epoch:  26
2017-11-15T22:51:16.135349: step 470, loss 0.841944, acc 0.571289, f1 0.514248
2017-11-15T22:51:16.649780: step 475, loss 0.682645, acc 0.702148, f1 0.695522
2017-11-15T22:51:17.172266: step 480, loss 0.679527, acc 0.695312, f1 0.659669
2017-11-15T22:51:17.677829: step 485, loss 0.659211, acc 0.74707, f1 0.748812
Current epoch:  27
2017-11-15T22:51:18.154199: step 490, loss 0.802403, acc 0.585938, f1 0.533702
2017-11-15T22:51:18.665154: step 495, loss 0.705178, acc 0.638672, f1 0.616745
2017-11-15T22:51:19.178081: step 500, loss 0.665066, acc 0.707031, f1 0.694725

Evaluation:
2017-11-15T22:51:19.813875: step 500, loss 0.802126, acc 0.622

2017-11-15T22:52:04.343164: step 895, loss 0.556699, acc 0.762695, f1 0.734542
2017-11-15T22:52:04.830258: step 900, loss 0.528706, acc 0.783019, f1 0.779091

Evaluation:
2017-11-15T22:52:05.472223: step 900, loss 0.891266, acc 0.611865, f1 0.576822

Current epoch:  50
2017-11-15T22:52:05.985842: step 905, loss 0.599252, acc 0.753906, f1 0.720575
2017-11-15T22:52:06.505884: step 910, loss 0.604552, acc 0.696289, f1 0.682843
2017-11-15T22:52:07.020471: step 915, loss 0.522283, acc 0.795898, f1 0.788998
Current epoch:  51
2017-11-15T22:52:07.501346: step 920, loss 0.460165, acc 0.806641, f1 0.794408
2017-11-15T22:52:08.011261: step 925, loss 0.550276, acc 0.738281, f1 0.725459
2017-11-15T22:52:08.527353: step 930, loss 0.540491, acc 0.77832, f1 0.772678
2017-11-15T22:52:09.036257: step 935, loss 0.464093, acc 0.807617, f1 0.792716
Current epoch:  52
2017-11-15T22:52:09.516615: step 940, loss 0.604742, acc 0.696289, f1 0.671039
2017-11-15T22:52:10.022247: step 945, loss 0.440876, acc 0.83

...

2017-11-15T23:42:21.058171: step 26930, loss 0.000576486, acc 1, f1 1
2017-11-15T23:42:21.577169: step 26935, loss 0.00287609, acc 0.999023, f1 0.999024
2017-11-15T23:42:22.111117: step 26940, loss 0.00217488, acc 0.998047, f1 0.998047
2017-11-15T23:42:22.644734: step 26945, loss 0.00109326, acc 0.999023, f1 0.999023
Current epoch:  1497
2017-11-15T23:42:23.139613: step 26950, loss 0.00110845, acc 0.999023, f1 0.999023

Evaluation:
2017-11-15T23:42:23.800927: step 26950, loss 2.26741, acc 0.584723, f1 0.584426

2017-11-15T23:42:24.316848: step 26955, loss 7.5028e-05, acc 1, f1 1
2017-11-15T23:42:24.846815: step 26960, loss 0.00286973, acc 0.99707, f1 0.99707
Current epoch:  1498
2017-11-15T23:42:25.337550: step 26965, loss 0.00252285, acc 0.998047, f1 0.998048
2017-11-15T23:42:25.866557: step 26970, loss 0.000978355, acc 0.999023, f1 0.999023
2017-11-15T23:42:26.385013: step 26975, loss 0.00251714, acc 0.999023, f1 0.999024
2017-11-15T23:42:26.910960: step 26980, loss 0.00014475, acc 1

2017-11-15T23:43:11.419766: step 27360, loss 0.00348864, acc 0.996855, f1 0.996852
Current epoch:  1520
2017-11-15T23:43:11.940259: step 27365, loss 0.00145678, acc 0.999023, f1 0.999023
2017-11-15T23:43:12.458188: step 27370, loss 0.00126055, acc 0.999023, f1 0.999023
2017-11-15T23:43:12.972219: step 27375, loss 0.0036591, acc 0.99707, f1 0.997074
Current epoch:  1521
2017-11-15T23:43:13.456052: step 27380, loss 0.00191984, acc 0.999023, f1 0.999023
2017-11-15T23:43:13.980553: step 27385, loss 0.000109907, acc 1, f1 1
2017-11-15T23:43:14.494472: step 27390, loss 0.000739372, acc 1, f1 1
2017-11-15T23:43:15.010433: step 27395, loss 0.00271221, acc 0.99707, f1 0.99707
Current epoch:  1522
2017-11-15T23:43:15.505023: step 27400, loss 0.00135726, acc 0.999023, f1 0.999022

Evaluation:
2017-11-15T23:43:16.171231: step 27400, loss 2.25063, acc 0.590927, f1 0.588404

2017-11-15T23:43:16.685637: step 27405, loss 0.00206267, acc 0.999023, f1 0.999023
2017-11-15T23:43:17.204554: step 27410, los

Current epoch:  1544
2017-11-15T23:44:06.140965: step 27795, loss 0.000815335, acc 0.999023, f1 0.999023
2017-11-15T23:44:06.683980: step 27800, loss 0.00224603, acc 0.998047, f1 0.998047

Evaluation:
2017-11-15T23:44:07.394014: step 27800, loss 2.25096, acc 0.590054, f1 0.588589

2017-11-15T23:44:07.947010: step 27805, loss 0.00508549, acc 0.995117, f1 0.995122
2017-11-15T23:44:08.446899: step 27810, loss 0.00545572, acc 0.995283, f1 0.995276
Current epoch:  1545
2017-11-15T23:44:08.973901: step 27815, loss 0.00169079, acc 0.999023, f1 0.999023
2017-11-15T23:44:09.502134: step 27820, loss 0.00111834, acc 1, f1 1
2017-11-15T23:44:10.023133: step 27825, loss 0.00559433, acc 0.995117, f1 0.995117
Current epoch:  1546
2017-11-15T23:44:10.521547: step 27830, loss 0.000777822, acc 0.999023, f1 0.999023
2017-11-15T23:44:11.094017: step 27835, loss 0.00130433, acc 0.999023, f1 0.999023
2017-11-15T23:44:11.658763: step 27840, loss 0.00370254, acc 0.99707, f1 0.997071
2017-11-15T23:44:12.206294

2017-11-15T23:44:57.876623: step 28220, loss 0.00408196, acc 0.996094, f1 0.99609
Current epoch:  1568
2017-11-15T23:44:58.362035: step 28225, loss 0.00356103, acc 0.998047, f1 0.998047
2017-11-15T23:44:58.879607: step 28230, loss 0.000769548, acc 0.999023, f1 0.999023
2017-11-15T23:44:59.399593: step 28235, loss 0.00142902, acc 1, f1 1
2017-11-15T23:44:59.920234: step 28240, loss 0.00287693, acc 0.99707, f1 0.99707
Current epoch:  1569
2017-11-15T23:45:00.442704: step 28245, loss 0.00191892, acc 0.998047, f1 0.998047
2017-11-15T23:45:00.977690: step 28250, loss 0.00151672, acc 0.999023, f1 0.999023

Evaluation:
2017-11-15T23:45:01.674934: step 28250, loss 2.27738, acc 0.589327, f1 0.587414

2017-11-15T23:45:02.229183: step 28255, loss 0.00299201, acc 0.99707, f1 0.997071
2017-11-15T23:45:02.709038: step 28260, loss 0.00192059, acc 0.998428, f1 0.998428
Current epoch:  1570
2017-11-15T23:45:03.293852: step 28265, loss 6.47381e-05, acc 1, f1 1
2017-11-15T23:45:03.863381: step 28270, los

2017-11-15T23:45:49.310977: step 28650, loss 2.2659, acc 0.591896, f1 0.590577

2017-11-15T23:45:49.825484: step 28655, loss 0.00386814, acc 0.996094, f1 0.996094
Current epoch:  1592
2017-11-15T23:45:50.310022: step 28660, loss 0.00153759, acc 0.999023, f1 0.999023
2017-11-15T23:45:50.822744: step 28665, loss 0.00258661, acc 0.998047, f1 0.998047
2017-11-15T23:45:51.352169: step 28670, loss 0.00137868, acc 0.999023, f1 0.999023
Current epoch:  1593
2017-11-15T23:45:51.842602: step 28675, loss 0.000639613, acc 1, f1 1
2017-11-15T23:45:52.360281: step 28680, loss 0.00294478, acc 0.99707, f1 0.997082
2017-11-15T23:45:52.875169: step 28685, loss 0.00427626, acc 0.99707, f1 0.997069
2017-11-15T23:45:53.398795: step 28690, loss 0.00130105, acc 0.999023, f1 0.999022
Current epoch:  1594
2017-11-15T23:45:53.891669: step 28695, loss 0.00191334, acc 0.999023, f1 0.999024
